In [89]:
import deepcut
import numpy as np
from tensorflow.keras.preprocessing import sequence
from tensorflow import keras

In [90]:
x_train = ['วันนี้วันจันทร์','วันนี้วันศุกร์','วันนี้วันเสาร์','วันนีวันจันทนะ','วันนีวันศุกนะ','วันนีวันศุร์นะ','วันนี้วันศุกร์นะ','วันนี้วันจันทร์นะ',]
y_train = ['ถูก','ถูก','ถูก','ผิด','ผิด','ผิด','ถูก','ถูก',]
x_test = ['วันนี้วันพุธ','วันนี้วันอังคาร','วันนี้วันอังคาน',]

In [92]:
corpus = ['วันนี้วันจันทร์','วันนี้วันศุกร์','วันนี้วันเสาร์','วันนีวันจันทนะ','วันนีวันศุกนะ',
          'วันนีวันศุร์นะ','วันนี้วันศุกร์นะ','วันนี้วันจันทร์นะ','วันนี้วันพุธ','วันนี้วันอังคาร','วันนี้วันอังคาน',]

In [47]:
test = deepcut.tokenize('วันนี้วันศุร์นะ')
print(test)

['วัน', 'นี้', 'วัน', 'ศุร์นะ']


In [93]:
def word_tokenize(corpus):
    x = 0
    words = []
    for i in corpus:
        list_word = deepcut.tokenize(i,custom_dict='custom_dict.txt') #ตัดคำ
        words.append(list_word)
        x += 1
    return words
corpus=word_tokenize(corpus)
words = word_tokenize(x_train)
print(words)
print(len(words))

[['วัน', 'นี้', 'วัน', 'จันทร์'], ['วัน', 'นี้', 'วัน', 'ศุกร์'], ['วัน', 'นี้', 'วัน', 'เสาร์'], ['วัน', 'นีวัน', 'จันทนะ'], ['วัน', 'นีวัน', 'ศุก', 'นะ'], ['วัน', 'นีวัน', 'ศุร์นะ'], ['วัน', 'นี้', 'วัน', 'ศุกร์', 'นะ'], ['วัน', 'นี้', 'วัน', 'จันทร์', 'นะ']]
8


In [103]:
def add_1list(word_list):
    list_ = []
    vocab = []
    for l in word_list:
        list_.extend(l)
#     print(list_)
    word_v = set(list_)
    for list_d in word_v:
        vocab.append(list_d)
    vocab = [' '] + vocab
    return vocab

vocab = add_1list(corpus)
print(vocab)

[' ', 'นะ', 'เสาร์', 'วัน', 'จันทนะ', 'นี้', 'ศุกร์', 'ศุก', 'อังคาน', 'นีวัน', 'พุธ', 'อังคาร', 'จันทร์', 'ศุร์นะ']


In [104]:
words1 = []
x = 0
y = 0
for text in range(len(words)):
    for word in range(len(words[y])):
        words1.append(words[y][x])
        x += 1
    y += 1
    x = 0
print(words1)
words1 = set(words1)
print(words1)
# vocab = []
# for list_d in words1:
#     vocab.append(list_d)
# vocab = [' '] + vocab
# print(vocab)
# print(len(vocab))

['วัน', 'นี้', 'วัน', 'จันทร์', 'วัน', 'นี้', 'วัน', 'ศุกร์', 'วัน', 'นี้', 'วัน', 'เสาร์', 'วัน', 'นีวัน', 'จันทนะ', 'วัน', 'นีวัน', 'ศุก', 'นะ', 'วัน', 'นีวัน', 'ศุร์นะ', 'วัน', 'นี้', 'วัน', 'ศุกร์', 'นะ', 'วัน', 'นี้', 'วัน', 'จันทร์', 'นะ']
{'นะ', 'เสาร์', 'วัน', 'จันทนะ', 'นี้', 'ศุกร์', 'ศุก', 'นีวัน', 'จันทร์', 'ศุร์นะ'}


In [105]:
def sent2idx(x):
    xidx = []
    for w in x:
#         print(w)
        if w in vocab:
            xidx.append(vocab.index(w))
    return np.array(xidx)

In [106]:
sent2idx(words[0])

array([ 3,  5,  3, 12])

In [107]:
def word_seq(words):
    Xtrain_idx = []
    maxlen = 0
    for x in words:
        Xtrain_idx.append(sent2idx(x))
        if len(Xtrain_idx[-1]) > maxlen:
            maxlen = len(Xtrain_idx[-1])
#         print(len(Xtrain_idx[-1]))
    return Xtrain_idx

In [108]:
Xtrain = word_seq(words)
print(Xtrain)

[array([ 3,  5,  3, 12]), array([3, 5, 3, 6]), array([3, 5, 3, 2]), array([3, 9, 4]), array([3, 9, 7, 1]), array([ 3,  9, 13]), array([3, 5, 3, 6, 1]), array([ 3,  5,  3, 12,  1])]


In [109]:
Xtrain = sequence.pad_sequences(np.array(Xtrain),padding='pre')
print(Xtrain)

[[ 0  3  5  3 12]
 [ 0  3  5  3  6]
 [ 0  3  5  3  2]
 [ 0  0  3  9  4]
 [ 0  3  9  7  1]
 [ 0  0  3  9 13]
 [ 3  5  3  6  1]
 [ 3  5  3 12  1]]


In [110]:
Xtrain.shape

(8, 5)

In [111]:
intput_lenght = len(Xtrain[0])
print(intput_lenght)
vocab_size = len(vocab)+1

5


In [136]:
model = keras.Sequential()
model.add(keras.layers.Embedding(input_dim = vocab_size,output_dim = 300,input_length=intput_lenght))
model.add(keras.layers.Bidirectional(keras.layers.LSTM(8, return_sequences=True)))
model.add(keras.layers.Bidirectional(keras.layers.LSTM(4)))
model.add(keras.layers.Dense(32, activation=keras.activations.relu))
model.add(keras.layers.Dense(2))

In [137]:
model.compile(loss=keras.losses.sparse_categorical_crossentropy, optimizer=keras.optimizers.SGD(learning_rate=1e-3))

In [138]:
print(model.summary())

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, 5, 300)            4500      
_________________________________________________________________
bidirectional_22 (Bidirectio (None, 5, 16)             19776     
_________________________________________________________________
bidirectional_23 (Bidirectio (None, 8)                 672       
_________________________________________________________________
dense_36 (Dense)             (None, 32)                288       
_________________________________________________________________
dense_37 (Dense)             (None, 2)                 66        
Total params: 25,302
Trainable params: 25,302
Non-trainable params: 0
_________________________________________________________________
None


In [142]:
model.fit(Xtrain, np.array([1,1,1,0,0,0,1,1]), epochs=10)

Epoch 1/10
1/1 [==============================] - 0s 2ms/step - loss: 0.6931
Epoch 2/10
1/1 [==============================] - 0s 1ms/step - loss: 0.6931
Epoch 3/10
1/1 [==============================] - 0s 2ms/step - loss: 0.6931
Epoch 4/10
1/1 [==============================] - 0s 2ms/step - loss: 0.6931
Epoch 5/10
1/1 [==============================] - 0s 2ms/step - loss: 0.6931
Epoch 6/10
1/1 [==============================] - 0s 1ms/step - loss: 0.6931
Epoch 7/10
1/1 [==============================] - 0s 1ms/step - loss: 0.6931
Epoch 8/10
1/1 [==============================] - 0s 1000us/step - loss: 0.6931
Epoch 9/10
1/1 [==============================] - 0s 2ms/step - loss: 0.6931
Epoch 10/10
1/1 [==============================] - 0s 1ms/step - loss: 0.6931


In [143]:
model.predict(Xtrain).argmax(axis=1)

array([1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)